In [ ]:
import torch

# CUDA major, minor 버전 확인
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

# unsloth 설치
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-92re6uxy/unsloth_95ed79c3cf954a9d84c69bcc5a664d61
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-92re6uxy/unsloth_95ed79c3cf954a9d84c69bcc5a664d61
  Resolved https://github.com/unslothai/unsloth.git to commit 937f684d4377c465452a7723c8bb97f1ecd2a3d5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 102.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 79.1 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [ ]:
!pip install langchain
!pip install langchain-community
!pip install pymupdf
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 76.1 MB/s eta 0:00:00


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("SPRi AI Brief 5월호 산업동향.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
        model_name="BM-K/KoSimCSE-roberta-multitask"
        # model_name = 'jhgan/ko-sroberta-nli'
    )

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """다음 컨텍스트를 사용하여 질문에 답변하세요.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델명 설정
    max_seq_length=16384,  # 최대 시퀀스 길이 설정
    dtype=None,  # 데이터 타입을 설정
    load_in_4bit=True,  # 4bit 양자화 로드 여부 설정
)
# model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id
    # device=0,  # CPU 사용, GPU 사용 시 0 또는 적절한 디바이스 ID
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Device set to use cuda:0


In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "EU집행위원회는 AI에 얼마를 투자했지?"
response = chain.invoke(question)
print(response)

다음 컨텍스트에서 질문에 대한 답변을 찾아 1문장의 한국어로 작성하세요.
컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
[Document(id='ca8e0293-d2e4-4c9f-a9a5-0a08408d5328', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-05-09T09:07:04+09:00', 'source': 'SPRi AI Brief 5월호 산업동향.pdf', 'file_path': 'SPRi AI Brief 5월호 산업동향.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-05-09T09:07:04+09:00', 'trapped': '', 'modDate': "D:20250509090704+09'00'", 'creationDate': "D:20250509090704+09'00'", 'page': 4}, page_content='정책･법제\n기업･산업\n기술･연구\n인력･교육\n3\nEU 집행위원회, EU의 AI 리더십을 위한 AI 대륙 행동계획 발표\nn EU 집행위원회가 EU의 AI 리더십을 확보하기 위한 ‘AI 대륙 행동계획’을 수립하고 \n‘InvestAI’ 이니셔티브를 통해 AI에 총 2,000억 유로를 투자할 계획\nn AI 대륙 행동계획은 △AI 컴퓨팅 인프라 구축 △데이터 접근성 확대 △전략적 영역의 AI 촉진 \n△AI 역량과 인재 육성 △AI 법 시행 간소화의 5개 영역을 중점 추진\nKEY Contents\n£ AI 대륙 행동계획, 컴퓨팅 인프라와 데이터 접근성 확보 및 AI 도입 확대를 추진\nn EU 집행위원회가 2025년 4월 9일 EU의 AI 리더십을 확보하기 위해 중점 추진할 5개 핵심 영역을 \n제시한 

In [ ]:
question = "구글이 발표한 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜은?"
response = chain.invoke(question)
print(response)

다음 컨텍스트에서 질문에 대한 가장 핵심적인 답변을 찾으세요.
답변은 간결하고 정확하게 한국어로 작성하세요.
동일한 문장이나 내용을 절대 반복하지 마세요. 답변은 한 번만 명확하게 제시해야 합니다.
컨텍스트에 없는 정보는 사용하지 마세요.

#Context:
[Document(id='a8d64211-3fe4-424e-a8aa-7a41d156cdc6', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-05-09T09:07:04+09:00', 'source': 'SPRi AI Brief 5월호 산업동향.pdf', 'file_path': 'SPRi AI Brief 5월호 산업동향.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-05-09T09:07:04+09:00', 'trapped': '', 'modDate': "D:20250509090704+09'00'", 'creationDate': "D:20250509090704+09'00'", 'page': 10}, page_content='정책･법제\n기업･산업\n기술･연구\n인력･교육\n9\n구글, AI 에이전트 간 통신 프로토콜 ‘A2A’ 공개 및 MCP 지원 발표\nn 구글이 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜 A2A를 공개했으며, \nA2A는 에이전트 간 기능 탐색, 작업 관리, 협업, 사용자 경험 협의 등의 다양한 기능을 지원\nn 구글은 제미나이 모델과 SDK에서 앤스로픽의 MCP 지원을 추가하기로 했으며, A2A가 MCP보다 \n상위 계층의 프로토콜로서 MCP를 보완한다고 설명\nKEY Contents\n£ A2A, 다중 에이전트 간 협업을 위한 개방형 프로토콜로 설계\nn 구글(G